In [ ]:
import tensorflow as tf
import scipy.io.wavfile as wav
from tensorflow.python.lib.io import file_io
import numpy as np

def decode_csv(line):

    parsed_line = tf.decode_csv(line, [['string'], ['string']])
    feat = parsed_line[0]
    lab = parsed_line[-1]

    return feat, lab

def audiofile_to_input_vector(truth_file):

#    Load wav files (1st column)
#    binary_truth = file_io.FileIO(truth_file, 'b')
    fs_truth, audio_truth = wav.read(truth_file)    
    return audio_truth.astype( np.float32 ).reshape(-1,1) 


def input_parser(truth_file, ds_file):
 
    truth_audio_array = tf.py_func(audiofile_to_input_vector,[truth_file], [tf.float32])
    ds_audio_array = tf.py_func(audiofile_to_input_vector,[ds_file], [tf.float32])     
    
    return truth_audio_array[0] , ds_audio_array[0] 


# Training pipeline

def train_input(data_filepath, batch_size, num_epoch):
   

#   num_parallel_calls = cpu_count()
    dataset = tf.data.TextLineDataset(data_filepath).map(decode_csv, num_parallel_calls = 1)
    dataset = dataset.map(input_parser,num_parallel_calls=4)
#    dataset = dataset.apply(tf.contrib.data.ignore_errors())
    dataset = dataset.repeat(num_epoch)
    dataset = dataset.padded_batch( batch_size, padded_shapes=( [None,1], [None,1] ) )
#     dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    upsampled, downsampled = iterator.get_next()
    return upsampled, downsampled


# Inference pipeline
def decode_csv_inference(line):

    parsed_line = tf.decode_csv(line, [['string']])
    wave_name = parsed_line[0]

    return wave_name



def audiofile_to_input_vector_inference(truth_file):

#    Load wav files (1st column)
#    binary_truth = file_io.FileIO(truth_file, 'b')
    fs_truth, audio_truth = wav.read(truth_file)    
    return audio_truth.astype( np.float32 ).reshape(-1,1) 


def input_parser_inference(truth_file):
 
    truth_audio_array = tf.py_func(audiofile_to_input_vector_inference,[truth_file], [tf.float32])    
    
    return truth_audio_array[0]


# Training pipeline

def inference_input(data_filepath):
   

#   num_parallel_calls = cpu_count()
    dataset = tf.data.TextLineDataset(data_filepath).map(decode_csv_inference, num_parallel_calls = 1)
    dataset = dataset.map(input_parser_inference,num_parallel_calls=4)
    iterator = dataset.make_one_shot_iterator()
    waveform = iterator.get_next()
    return waveform

In [ ]:
filepath = '/home/mohit/Desktop/Research papers/Implementation/SpeechUpsampler_GPU/data/upsampling/upsampling_data/upsampling.csv'

waveform = inference_input(filepath)

    

In [ ]:
with tf.Session() as sess:
    for i in range(10):
        try:
            array = sess.run(waveform)
            print(array.shape)
        except tf.errors.OutOfRangeError as e:
            print(e)

In [3]:
import numpy as np
import os
import json
import librosa
import tensorflow as tf
import losses
from optimizers import make_variable_learning_rate, setup_optimizer
import models_exp
import pandas as pd
import datetime
import pipeline
import time


import scipy.io.wavfile as wav


upsampling_factor = 2

upsampling_path = './data/upsampling/upsampling_data/'


#custom_shuffle_module = tf.load_op_library('src/shuffle_op.so')
#shuffle = custom_shuffle_module.shuffle

try:
    os.makedirs('./aux/checkpoint')
except Exception: 
    pass

data_settings_file = 'settings/data_settings.json'
training_settings_file = 'settings/training_settings.json'
model_settings_file = 'settings/model_settings.json'

data_settings = json.load(open(data_settings_file))
training_settings = json.load(open(training_settings_file))
model_settings = json.load(open(model_settings_file))

# Constants describing the training process.
# Samples per batch.
BATCH_SIZE = training_settings['batch_size']
# Number of epochs to train.
NUMBER_OF_EPOCHS = training_settings['number_of_epochs']
# Epochs after which learning rate decays.
NUM_EPOCHS_PER_DECAY = training_settings['num_epochs_per_decay']
# Learning rate decay factor.
LEARNING_RATE_DECAY_FACTOR = training_settings['learning_rate_decay_factor']
# Initial learning rate.
INITIAL_LEARNING_RATE = training_settings['initial_learning_rate']

example_number = 0
write_tb = False

file_name_lists_dir = data_settings['output_dir_name_base']
train_filepath =  './data/preprocessed/train_files.csv'
validation_filepath = './data/preprocessed/validation_files.csv'



with tf.Graph().as_default():
    global_step = tf.train.get_or_create_global_step()
    with tf.Session() as sess:
        x = tf.placeholder(dtype = tf.float32,shape = [None,16000,1])
        train_flag, x , up_audio_return_y = models_exp.train( input_data = x, 
                                                        input_shape = x.get_shape())
        
        
        def read_file(wave, path):
            br, array = wav.read(path + wave)
            array = librosa.core.resample(array.astype(np.float32), br, br*upsampling_factor)
            array = np.pad(array,(0,16000-len(array)%16000), 'constant').reshape(-1,16000,1)
            array_tf = tf.convert_to_tensor(array)
            return array_tf


        for file in os.listdir(upsampling_path):
            if file.endswith('wav'):
                array_tf = read_file(file, upsampling_path)
        
        print('Model created')
        saver = tf.train.Saver()
        saver.restore(sess,'./aux/checkpoint/checkpoint1.ckpt-0')
        
        output = sess.run([up_audio_return_y], feed_dict = {x:array_tf})
        
        
        


layer summary for deep_residual network
Input: (?, 16000, 1)
bottleneck layer: [29, 2048]
restack layer: [8002, 5]
final conv layer: [8000, 2]
output: [16000, 1]
Model created
INFO:tensorflow:Restoring parameters from ./aux/checkpoint/checkpoint1.ckpt-0


NotFoundError: Key deep_residual/10_layer_batch_norm//beta not found in checkpoint
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "/home/mohit/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mohit/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mohit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mohit/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/mohit/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477,

In [3]:
import tensorflow as tf
import scipy.io.wavfile as wav
from tensorflow.python.lib.io import file_io
from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio
from optimizers import make_variable_learning_rate, setup_optimizer


import tensorflow as tf



# ######################
# LAYER HELPER FUNCTIONS
# ######################

def subpixel_reshuffle_1D_impl(X, m):
    """
    performs a 1-D subpixel reshuffle of the input 2-D tensor
    assumes the last dimension of X is the filter dimension
    ref: https://github.com/Tetrachrome/subpixel
    """
    return tf.transpose(tf.stack([tf.reshape(x, (-1,)) for x
                                  in tf.split(X, m, axis=1)]))


def subpixel_reshuffle_1D(X, m, name=None):
    """
    maps over the batch dimension
    """
    return tf.map_fn(lambda x: subpixel_reshuffle_1D_impl(x, m), X, name=name)


def subpixel_restack_impl(X, n_prime, m_prime, name=None):
    """
    performs a subpixel restacking such that it restacks columns of a 2-D
    tensor onto the rows
    """
    bsize = tf.shape(X)[0]
    r_n = n_prime - X.get_shape().as_list()[1]
    total_new_space = r_n*m_prime
    to_stack = tf.slice(X, [0, 0, m_prime], [-1, -1, -1])
    to_stack = tf.slice(tf.reshape(to_stack, (bsize, -1)),
                        [0, 0], [-1, total_new_space])
    to_stack = tf.reshape(to_stack, (bsize, -1, m_prime))
    to_stack = tf.slice(to_stack, [0, 0, 0], [-1, r_n, -1])
    return tf.concat((tf.slice(X, [0, 0, 0], [-1, -1, m_prime]), to_stack),
                     axis=1, name=name)


def subpixel_restack(X, n_prime, m_prime=None, name=None):
    n = X.get_shape().as_list()[1]
    m = X.get_shape().as_list()[2]
    r_n = n_prime - n
    if m_prime is None:
        for i in range(1, m):
            r_m = i
            m_prime = m - r_m
            if r_m*n >= m_prime*r_n:
                break
    return subpixel_restack_impl(X, n_prime, m_prime, name=name)


def batch_norm(T, is_training, scope):
    # tf.cond takes nullary functions as its first and second arguments
    return tf.cond(is_training,
                   lambda: tf.contrib.layers.batch_norm(T,
                                                decay=0.99,
                                                # zero_debias_moving_mean=True,
                                                is_training=is_training,
                                                center=True, scale=True,
                                                updates_collections=None,
                                                scope=scope,
                                                reuse=False),
                   lambda: tf.contrib.layers.batch_norm(T,
                                                decay=0.99,
                                                is_training=is_training,
                                                center=True, scale=True,
                                                updates_collections=None,
                                                scope=scope,
                                                reuse=True))


def weight_variable(shape, name=None):
    initial = tf.truncated_normal(shape, mean=0.0, stddev=0.1)
    return tf.Variable(initial, name=name)


def bias_variable(shape, name=None):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)


def conv1d(x, W, stride=1, padding='SAME', name=None):
    return tf.nn.conv1d(x, W, stride=stride, padding=padding, name=name)


def build_1d_conv_layer(prev_tensor, prev_conv_depth,
                        conv_window, conv_depth,
                        act, layer_number,
                        stride=1,
                        padding='SAME',
                        tensorboard_output=False,
                        name=None):
    with tf.name_scope('{}_layer_weights'.format(layer_number)):
        W = weight_variable([conv_window,
                             prev_conv_depth,
                             conv_depth])
        if tensorboard_output:
            histogram_variable_summaries(W)
    with tf.name_scope('{}_layer_biases'.format(layer_number)):
        b = bias_variable([conv_depth])
        if tensorboard_output:
            histogram_variable_summaries(b)
    with tf.name_scope('{}_layer_conv_preactivation'.format(layer_number)):
        conv = conv1d(prev_tensor, W, stride=stride, padding=padding) + b
        if tensorboard_output:
            histogram_variable_summaries(conv)
    with tf.name_scope('{}_layer_conv_activation'.format(layer_number)):
        h = act(conv, name=name)
        if tensorboard_output:
            histogram_variable_summaries(h)
    return h


def build_downsampling_block(input_tensor,
                             filter_size, stride,
                             layer_number,
                             act=tf.nn.relu,
                             is_training=True,
                             depth=None,
                             padding='VALID',
                             tensorboard_output=False,
                             name=None):

    # assume this layer is twice the depth of the previous layer if no depth
    # information is given
    
    if depth is None:
        depth = 2*input_tensor.get_shape().as_list()[-1]

    with tf.name_scope('{}_layer_weights'.format(layer_number)):
        W = weight_variable([filter_size,
                             input_tensor.get_shape().as_list()[-1],
                             depth])
        if tensorboard_output:
            histogram_variable_summaries(W)
    with tf.name_scope('{}_layer_biases'.format(layer_number)):
        b = bias_variable([depth])
        if tensorboard_output:
            histogram_variable_summaries(b)
    with tf.name_scope('{}_layer_conv_preactivation'.format(layer_number)):
        l = tf.nn.conv1d(input_tensor, W, stride=stride,
                         padding=padding, name=name) + b
        if tensorboard_output:
            histogram_variable_summaries(l)
#     with tf.name_scope('{}_layer_batch_norm'.format(layer_number)) as scope:
#         # l = tf.nn.dropout(l, keep_prob=0.25)
#         l = batch_norm(l, is_training, scope)
    with tf.name_scope('{}_layer_conv_activation'.format(layer_number)):
        l = act(l, name=name)
        if tensorboard_output:
            histogram_variable_summaries(l)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print('Downsample layer shape',sess.run(l).shape)
    return l


def build_upsampling_block(input_tensor, residual_tensor,
                           filter_size,
                           layer_number,
                           act=tf.nn.relu,
                           is_training=True,
                           depth=None,
                           padding='VALID',
                           tensorboard_output=False,
                           name=None):

    # assume this layer is half the depth of the previous layer if no depth
    # information is given
    if depth is None:
        depth = int(input_tensor.get_shape().as_list()[-1]/2)

    with tf.name_scope('{}_layer_weights'.format(layer_number)):
        W = weight_variable([filter_size,
                             input_tensor.get_shape().as_list()[-1],
                             depth])
        if tensorboard_output:
            histogram_variable_summaries(W)
    with tf.name_scope('{}_layer_biases'.format(layer_number)):
        b = bias_variable([depth])
        if tensorboard_output:
            histogram_variable_summaries(b)
    with tf.name_scope('{}_layer_conv_preactivation'.format(layer_number)):
        l = tf.nn.conv1d(input_tensor, W, stride=1,
                         padding=padding, name=name) + b
        if tensorboard_output:
            histogram_variable_summaries(l)
#     with tf.name_scope('{}_layer_batch_norm'.format(layer_number)) as scope:
#         # l = tf.nn.dropout(l, keep_prob=0.25)
#         l = batch_norm(l, is_training, scope)
        # l = tf.nn.l2_normalize(l, dim=2)
    with tf.name_scope('{}_layer_conv_activation'.format(layer_number)):
        l = act(l, name=name)
        if tensorboard_output:
            histogram_variable_summaries(l)
    with tf.name_scope('{}_layer_subpixel_reshuffle'.format(layer_number)):
        l = subpixel_reshuffle_1D(l,
                                  residual_tensor.get_shape().as_list()[-1],
                                  name=name)
        if tensorboard_output:
            histogram_variable_summaries(l)
    with tf.name_scope('{}_layer_stacking'.format(layer_number)):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            l_eval = sess.run(l)
            print('Upsampler layer shape and other', l_eval.shape, l_eval.shape[1])
        sliced = tf.slice(residual_tensor,
                          begin=[0, 0, 0],
                          size=[-1, l_eval.shape[1], -1])
        l = tf.concat((l, sliced), axis=2, name=name)
        if tensorboard_output:
            histogram_variable_summaries(l)

    return l

# ######################
# ######################


# #################
# MODEL DEFINITIONS
# #################


def train(input_data , input_shape,
                          number_of_downsample_layers=8,
                          channel_multiple=8,
                          initial_filter_window=5,
                          initial_stride=2,
                          downsample_filter_window=3,
                          downsample_stride=2,
                          bottleneck_filter_window=4,
                          bottleneck_stride=2,
                          upsample_filter_window=3,
                          tensorboard_output=False,
                          scope_name='deep_residual'):

    print('layer summary for {} network'.format(scope_name))
    downsample_layers = []
    upsample_layers = []

    with tf.name_scope(scope_name):
        # training flag
        train_flag =  tf.cast(1,tf.bool)

        # input of the model (examples)
        x = input_data
        input_size = input_shape[-2]
        num_of_channels = input_shape[-1]
        print('input: {}'.format(input_shape))

        d1 = build_downsampling_block(x,
                                      filter_size=initial_filter_window,
                                      stride=initial_stride,
                                      tensorboard_output=tensorboard_output,
                                      depth=channel_multiple*num_of_channels,
                                      is_training=train_flag,
                                      layer_number=1)
        downsample_layers.append(d1)

        layer_count = 2
        for i in range(number_of_downsample_layers - 1):
            d = build_downsampling_block(
                downsample_layers[-1],
                filter_size=downsample_filter_window,
                stride=downsample_stride,
                tensorboard_output=tensorboard_output,
                is_training=train_flag,
                layer_number=layer_count)
#             print('downsample layer: {}'.format(d.get_shape().as_list()[1:]))
            downsample_layers.append(d)
            layer_count += 1

        bn = build_downsampling_block(downsample_layers[-1],
                                      filter_size=bottleneck_filter_window,
                                      stride=bottleneck_stride,
                                      tensorboard_output=tensorboard_output,
                                      is_training=train_flag,
                                      layer_number=layer_count)
        print('bottleneck layer: {}'.format(bn.get_shape().as_list()[1:]))
        layer_count += 1
        

        u1 = build_upsampling_block(bn, downsample_layers[-1],
                                    depth=bn.get_shape().as_list()[-1],
                                    filter_size=upsample_filter_window,
                                    tensorboard_output=tensorboard_output,
                                    is_training=train_flag,
                                    layer_number=layer_count)
        upsample_layers.append(u1)
        layer_count += 1

        for i in range(number_of_downsample_layers - 2, -1, -1):
            u = build_upsampling_block(upsample_layers[-1],
                                       downsample_layers[i],
                                       filter_size=upsample_filter_window,
                                       tensorboard_output=tensorboard_output,
                                       is_training=train_flag,
                                       layer_number=layer_count)
            upsample_layers.append(u)
            layer_count += 1

        target_size = int(input_size/initial_stride)
        restack = subpixel_restack(upsample_layers[-1],
                                   target_size + (upsample_filter_window - 1))
        print('restack layer: {}'.format(restack.get_shape().as_list()[1:]))

        conv = build_1d_conv_layer(restack, restack.get_shape().as_list()[-1],
                                   upsample_filter_window, initial_stride,
                                   tf.nn.elu, layer_count,
                                   padding='VALID',
                                   tensorboard_output=tensorboard_output)
        print('final conv layer: {}'.format(conv.get_shape().as_list()[1:]))

        # NOTE this effectively is a linear activation on the last conv layer
        y = subpixel_reshuffle_1D(conv,
                                  num_of_channels)
        y = tf.add(y, x, name=scope_name)
        print('output: {}'.format(y.get_shape().as_list()[1:]))

    return train_flag, x , y


ValueError: No variables to save

In [ ]:
import os
import json
import numpy as np
import librosa
import tensorflow as tf
import pandas as pd
import models_inference as mi


data_settings_file = 'settings/data_settings.json'
model_settings_file = 'settings/model_settings.json'
upsampling_settings_file = 'settings/upsampling_settings.json'

data_settings = json.load(open(data_settings_file))
model_settings = json.load(open(model_settings_file))
upsampling_settings = json.load(open(upsampling_settings_file))

upsample_csv = upsampling_settings['input_file']+ upsampling_settings['filename']

source_dir = os.path.split(upsample_csv)[0]


END_OFFSET = data_settings['end_time']
upsampling_factor = 2
INPUT_SIZE = int(data_settings['downsample_rate'])*upsampling_factor



model_checkpoint_file_name = os.getcwd() + upsampling_settings['model_checkpoint_file']


# Iterate through complete upsampling list and convert
df = pd.read_csv(upsample_csv, header = None)

true_wf, true_br = librosa.load(df.iloc[0,0], sr=None, mono=True)

us_wf = librosa.core.resample(true_wf, true_br, true_br*upsampling_factor)
us_br = true_br*upsampling_factor

train_flag, x, model = mi.deep_residual_network(true_wf.dtype,
                                                 np.reshape(us_wf, (-1,1))[:us_br].shape,
                                                 **model_settings)


saver = tf.train.Saver()

corrupt_file = []
with tf.Session() as sess:
        # create session and restore model
    saver.restore(sess, model_checkpoint_file_name)

    for index in range(len(df)):
        # Load 8k file
        true_wf, true_br = librosa.load(df.iloc[index,0], sr=None, mono=True)

        # Upsampled file. This will be fed into Deep Neural Network
        us_wf = librosa.core.resample(true_wf, true_br, true_br*upsampling_factor)
        us_br = true_br*upsampling_factor
        number_of_reco_iterations = int(us_wf.size/INPUT_SIZE)



        # ###################
        # RECONSTRUCTION LOOP
        # ###################

        reco_wf = np.empty(us_wf.size)
        print('Processing:',df.iloc[index,0])
        # Calculate number of samples in test file
        for i in range(number_of_reco_iterations):
            print('Segement {} of {}'.format(i + 1, number_of_reco_iterations))
        # Feeding 
            example = us_wf[i*INPUT_SIZE:(i + 1)*INPUT_SIZE]
            reco_wf[i*INPUT_SIZE:(i + 1)*INPUT_SIZE] = model.eval(feed_dict={train_flag: False,
                                                                             x: example.reshape(1, -1, 1)},
                                                                  session=sess).flatten()

        file_name_base = df.iloc[index,0].split('/')[-1]
        
        try:
            librosa.output.write_wav(os.path.join(source_dir, 'reco_' + file_name_base),y=reco_wf, sr=us_br)
        except Exception:
            corrupt_file.append(df.iloc[index,0])
            pass
        
df = pd.DataFrame(corrupt_file)
corrupt_file.to_csv('currupt_files.csv', header = None, index = False)

In [ ]:
import scipy.io.wavfile as wav
import path
import os
import tensorflow as tf
import numpy as np
import librosa
import pandas as pd


In [ ]:
import numpy as np
import os
import json
import librosa
import tensorflow as tf
import losses
from optimizers import make_variable_learning_rate, setup_optimizer
import models
import models_exp
import pandas as pd
import datetime
import pipeline
import time
from tensorflow.python.saved_model import tag_constants

#custom_shuffle_module = tf.load_op_library('src/shuffle_op.so')
#shuffle = custom_shuffle_module.shuffle

try:
    os.makedirs('./aux/checkpoint')
except Exception: 
    pass

data_settings_file = 'settings/data_settings.json'
training_settings_file = 'settings/training_settings.json'
model_settings_file = 'settings/model_settings.json'

data_settings = json.load(open(data_settings_file))
training_settings = json.load(open(training_settings_file))
model_settings = json.load(open(model_settings_file))

# Constants describing the training process.
# Samples per batch.
BATCH_SIZE = training_settings['batch_size']
# Number of epochs to train.
NUMBER_OF_EPOCHS = training_settings['number_of_epochs']
# Epochs after which learning rate decays.
NUM_EPOCHS_PER_DECAY = training_settings['num_epochs_per_decay']
# Learning rate decay factor.
LEARNING_RATE_DECAY_FACTOR = training_settings['learning_rate_decay_factor']
# Initial learning rate.
INITIAL_LEARNING_RATE = training_settings['initial_learning_rate']

example_number = 0
write_tb = False

file_name_lists_dir = data_settings['output_dir_name_base']
train_filepath =  './data/preprocessed/train_files.csv'
validation_filepath = './data/preprocessed/validation_files.csv'

def read_csv(filepath):
    df = pd.read_csv(filepath)
    return len(df)



SAMPLES_PER_EPOCH_TRAIN = read_csv(train_filepath)
SAMPLES_PER_EPOCH_VALID = read_csv(validation_filepath)


 #### my code #### 
current_data = []



with tf.Graph().as_default():
    global_step = tf.train.get_or_create_global_step()
    saver = tf.train.Saver() 
    with tf.Session() as sess:
               
#         Training pipeline
        inp = pipeline.train_input( train_filepath, BATCH_SIZE, NUMBER_OF_EPOCHS )


        up_audio , down_audio = inp

        up_audio_eval =  sess.run(up_audio)
        
        print('Model called')
        train_flag, x , up_audio_return_y = models.train( input_data = up_audio , 
                                                                input_shape = up_audio_eval.shape)
        print('Model created')
#         Calculate loss.

        loss = losses.mse("loss",up_audio, up_audio_return_y)

#         Variable that affect learning rate.
        num_batches_per_epoch_train = int(SAMPLES_PER_EPOCH_TRAIN/BATCH_SIZE)
        print('num_batches_per_epoch:',num_batches_per_epoch_train)
        decay_steps = int(num_batches_per_epoch_train * NUM_EPOCHS_PER_DECAY)


        # Decay the learning rate based on the number of steps.
        lr, global_step = make_variable_learning_rate(INITIAL_LEARNING_RATE,
                                                      decay_steps,
                                                      LEARNING_RATE_DECAY_FACTOR,
                                                      False)

        min_args = {'global_step': global_step}

#             Defining optimizer
        train_step = setup_optimizer(lr, loss, tf.train.AdamOptimizer,
                                     using_batch_norm=True,
                                     min_args=min_args)
        
        
#             Training Loop
        try:
        
            sess.run(tf.global_variables_initializer())
            training_loss = open('train_loss.txt', 'w')
            valid_loss = open('validation_loss.txt', 'w')
            for i in range(NUMBER_OF_EPOCHS):
                for j in range(num_batches_per_epoch_train):
                    avg_loss = 0
                    try:
                        _, loss_value = sess.run([train_step, loss])
                        avg_loss +=  loss_value
                        if j % 50 == 0:
                            print( 'Training Loss in epoch {} and batch {} is {}:'.format(i+1,j+1, avg_loss/(j+1) ))
                    except tf.errors.OutOfRangeError :
                        print('Data used')
                        pass
                print( 'Training Loss in epoch {} is {}:'.format(i+1,avg_loss/(j+1) ))
                training_loss.write('Training Loss for Epoch {} is {}:\n'.format((i+1) , avg_loss/(j+1)))
                with open('trainable_weights.txt') as var_file:
                    var_file.write([v for v in tf.trainable_variables()])

    #         Validation pipeline
                validation = pipeline.train_input( validation_filepath, BATCH_SIZE, 1 )
                up_audio_valid, down_audio_valid = validation

                train_flag_val, x_val , up_audio_return_y_valid = models.train( input_data = 
                                                                                                up_audio_valid , 
                                                                        input_shape = up_audio_eval.shape)
                loss_valid = losses.mse("loss_valid",up_audio_valid, up_audio_return_y_valid)
                num_batches_per_epoch_valid = int(SAMPLES_PER_EPOCH_VALID/BATCH_SIZE)

                sess.run(tf.global_variables_initializer())
                for j in range(num_batches_per_epoch_valid):
                    avg_loss = 0
                    try:
                        loss_value = sess.run([loss_valid])
                        avg_loss +=  loss_value[0]

                    except tf.errors.OutOfRangeError :
                        print('Data used')
                        pass
                print('Validation loss for epoch {} is {}:'.format(i+1, avg_loss/(j+1)))
                valid_loss.write('Validation Loss for Epoch {} is {}:\n'.format((i+1) , avg_loss/(j+1)))
                checkpoint_path = saver.save(sess, './aux/checkpoint/checkpoint_valid{}.ckpt'.format(i+1),global_step=global_step)
                print('Checkpoint saved at:', checkpoint_path)
            training_loss.close()
            valid_loss.close()
        except Exception as e:
            print(e)
      

print('Process finished')



In [2]:
import tensorflow as tf
train_flag =  tf.cast(1,tf.bool)

/home/mohit/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
train_flag

<tf.Tensor 'Cast:0' shape=() dtype=bool>

In [4]:
with tf.Session() as sess:
    print(sess.run(train_flag))

True
